In [12]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

from zipfile import ZipFile
import os,time
for i in os.listdir('C:/Users/mehmu/Desktop/PyProjects/Projects/github repo/DiseaseDetection/TumorDataset'):
    if '.zip' in i:
        print(i)
        with ZipFile(i, 'r') as zipObj:
            zipObj.extractall()
        os.remove(i)


  0%|          | 0.00/15.1M [00:00<?, ?B/s]
  7%|▋         | 1.00M/15.1M [00:00<00:04, 3.32MB/s]
 20%|█▉        | 3.00M/15.1M [00:00<00:01, 8.71MB/s]
 40%|███▉      | 6.00M/15.1M [00:00<00:00, 14.8MB/s]
 60%|█████▉    | 9.00M/15.1M [00:00<00:00, 18.5MB/s]
 79%|███████▉  | 12.0M/15.1M [00:00<00:00, 21.0MB/s]
 99%|█████████▉| 15.0M/15.1M [00:00<00:00, 22.7MB/s]
100%|██████████| 15.1M/15.1M [00:00<00:00, 17.6MB/s]



brain-mri-images-for-brain-tumor-detection.zip


In [4]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow 


import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')

Num GPUs Available:  1


In [119]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.33, shear_range =.25, horizontal_flip = True, rotation_range=20)     


train_dataset = image_generator.flow_from_directory(batch_size=8,
                                                 directory='brain_tumor_dataset',
                                                 shuffle=True,
                                                 target_size=(512,512), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=8,
                                                 directory='brain_tumor_dataset',
                                                 shuffle=True,
                                                 target_size=(512,512), 
                                                 subset="validation",
                                                 class_mode='categorical')


Found 170 images belonging to 2 classes.
Found 83 images belonging to 2 classes.


In [120]:
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [512,512,3]),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(32, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),

keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),

keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),
keras.layers.Dropout(0.2),

keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),
keras.layers.Dropout(0.2),

keras.layers.Flatten(),

keras.layers.Dense(512, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.5),

keras.layers.Dense(128, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.5),

keras.layers.Dense(64, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.3),

keras.layers.Dense(2, activation ='softmax')])

model.compile(
        optimizer='adam',
        loss=tensorflow.losses.CategoricalCrossentropy(),
        metrics=[keras.metrics.AUC(name='auc'),'acc']
    )

In [121]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = keras.callbacks.ModelCheckpoint("tumor_model.h5",
                                                    save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_acc',patience=10,restore_best_weights=True)

In [122]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
    epochs=100
)

Epoch 1/100
22/22 [==============================] - 14s 557ms/step - loss: 0.9084 - auc: 0.6029 - acc: 0.5882 - val_loss: 4.3744 - val_auc: 0.6145 - val_acc: 0.6145 - lr: 0.0100
Epoch 2/100
22/22 [==============================] - 14s 643ms/step - loss: 0.7560 - auc: 0.7087 - acc: 0.6588 - val_loss: 1.3863 - val_auc: 0.7949 - val_acc: 0.7590 - lr: 0.0089
Epoch 3/100
22/22 [==============================] - 12s 554ms/step - loss: 0.6774 - auc: 0.6934 - acc: 0.6412 - val_loss: 0.8612 - val_auc: 0.7790 - val_acc: 0.7349 - lr: 0.0079
Epoch 4/100
22/22 [==============================] - 11s 525ms/step - loss: 0.6307 - auc: 0.7097 - acc: 0.6529 - val_loss: 22.2902 - val_auc: 0.3855 - val_acc: 0.3855 - lr: 0.0071
Epoch 5/100
22/22 [==============================] - 11s 507ms/step - loss: 0.7000 - auc: 0.6133 - acc: 0.5941 - val_loss: 1.2206 - val_auc: 0.6995 - val_acc: 0.6024 - lr: 0.0063
Epoch 6/100
22/22 [==============================] - 11s 495ms/step - loss: 0.6162 - auc: 0.7422 - acc: 

In [123]:
loss, auc, accuracy = model.evaluate(validation_dataset)
print("Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", accuracy)

11/11 [==============================] - 4s 305ms/step - loss: 1.4274 - auc: 0.7824 - acc: 0.7470
Loss:  1.427358627319336
AUC:  0.782406747341156
Accuracy:  0.7469879388809204


In [124]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0, shear_range =.25, horizontal_flip = True)


test = image_generator.flow_from_directory(batch_size=8,
                                                    directory='brain_tumor_dataset',
                                                    shuffle=True,
                                                    target_size=(512,512),
                                                    class_mode='binary')

Found 253 images belonging to 2 classes.


In [125]:
y_pred=np.argmax(model.predict(test),axis=1)

32/32 [==============================] - 11s 336ms/step


In [126]:
from sklearn.metrics import r2_score,accuracy_score,f1_score

r2_score(y_pred,test.classes)

-1.3753009147809343

In [127]:
accuracy_score(y_pred,test.classes)

0.5375494071146245

In [128]:
f1_score(y_pred,test.classes)

0.656891495601173